# Creación de cargadores de datos para un conjunto de datos de instrucciones

En la sección anterior, se introdujo una clase InstructionDataset y una función custom_collate_fn para el conjunto de datos de instrucciones.

Aprovechando el trabajo anterior se pueden conectar los objetos InstructionDataset y la función custom_collate_fn a los cargadores de datos de Pytorch. Estos  cargadores  reorganizarán  y  organizarán  automáticamente  los  lotes  para  el  proceso  de  ajuste  fino  de  las  instrucciones  LLM.

![Texto alternativo](./imgs/7.14.png)

La  función  custom_collate_fn  incluye  código  para  mover  los  tensores  de  entrada  y  de  destino  (por  ejemplo,  torch.stack(inputs_lst).to(device))  a  un  dispositivo  específico,  que  puede  ser  "cpu"  o  "cuda"  (para  GPU),  u  opcionalmente  "mps"  para  Mac  con  chips  Apple  Silicon.(A tener  en  cuenta  que  el  uso  de  un  dispositivo  "mps"  puede  generar  diferencias  numéricas  en  comparación  con  el  contenido  de  este  capítulo,  ya  que  la  compatibilidad  con  Apple  Silicon  en  PyTorch  aún  es  experimental).

En secciones anteriores, se movió  los  datos  al  dispositivo  de  destino  (por  ejemplo,  la  memoria  de  la  GPU  cuando  device="cuda")  en  el  bucle  de  entrenamiento  principal.  Incorporar  esto  como  parte  de  la  función  de  intercalación  ofrece  la  ventaja  de  realizar  este  proceso  de  transferencia  de  dispositivo  en  segundo  plano,  fuera  del  bucle  de  entrenamiento,  lo  que  evita  que  bloquee  la  GPU  durante  el  entrenamiento  del  modelo.

In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")                           
print("Device:", device)

Device: cuda


A  continuación,  para  reutilizar  la  configuración  del  dispositivo  seleccionada  en  custom_collate_fn  al  conectarla  a  la  clase  DataLoader  de  PyTorch  más  adelante  en  esta  sección,  usando  la  función  parcial  de  la  biblioteca  estándar  functools  de  Python  para  crear  una  nueva  versión  de  la  función  con  el  argumento  del  dispositivo  predefinido.  Además, se establece  el  valor  de  allowed_max_length  en  1024,  lo  que  trunca  los  datos  a  la  longitud  máxima  de  contexto  compatible  con  el  modelo  GPT2.

In [2]:
from custom_collocta import custom_collate_fn
from functools import partial
customized_collate_fn = partial(custom_collate_fn, device=device, allowed_max_length=1024)

A continuación, se puede configurar los cargadores de datos como se hizo anteriormente, pero esta vez se usará la función de intercalación personalizada para el proceso de procesamiento de lotes:

In [4]:
from instructionDataset import InstructionDataset
from torch.utils.data import DataLoader
import tiktoken
import pandas as pd
import json

num_workers = 0
batch_size = 8
tokenizer = tiktoken.get_encoding('gpt2')

with open('train.json', "r", encoding="utf-8") as file:
    train_data = json.load(file)

with open('val.json', "r", encoding="utf-8") as file:
    val_data = json.load(file)

with open('test.json', "r", encoding="utf-8") as file:
    test_data = json.load(file)
torch.manual_seed(123)

train_dataset = InstructionDataset(train_data, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)
val_dataset = InstructionDataset(val_data, tokenizer)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)
test_dataset = InstructionDataset(test_data, tokenizer)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

In [25]:
print("Train loader:")
for inputs, targets in train_loader:
    print(inputs.shape, targets.shape)

Train loader:
torch.Size([8, 60]) torch.Size([8, 60])
torch.Size([8, 63]) torch.Size([8, 63])
torch.Size([8, 75]) torch.Size([8, 75])
torch.Size([8, 69]) torch.Size([8, 69])
torch.Size([8, 67]) torch.Size([8, 67])
torch.Size([8, 71]) torch.Size([8, 71])
torch.Size([8, 75]) torch.Size([8, 75])
torch.Size([8, 68]) torch.Size([8, 68])
torch.Size([8, 67]) torch.Size([8, 67])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 72]) torch.Size([8, 72])
torch.Size([8, 70]) torch.Size([8, 70])
torch.Size([8, 76]) torch.Size([8, 76])
torch.Size([8, 77]) torch.Size([8, 77])
torch.Size([8, 61]) torch.Size([8, 61])
torch.Size([8, 65]) torch.Size([8, 65])
torch.Size([8, 69]) torch.Size([8, 69])
torch.Size([8, 71]) torch.Size([8, 71])
torch.Size([8, 74]) torch.Size([8, 74])
torch.Size([8, 62]) torch.Size([8, 62])
torch.Size([8, 69]) torch.Size([8, 69])
torch.Size([8, 69]) torch.Size([8, 69])
torch.Size([8, 68]) torch.Size([8, 68])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 59]) torch.

En  el  resultado  anterior, se puede ver  que  el  primer  lote  de  entrada  y  el  lote  de  destino  tienen  dimensiones  de  8×61,  donde  8  representa  el  tamaño  del  lote  y  61  el  número  de  tokens  en  cada  ejemplo  de  entrenamiento  de  este  lote.  El  segundo  lote  de  entrada  y  el  lote  de  destino  tienen  un  número  de  tokens  diferente;  por  ejemplo,  76.

Como  se vió  en  el  código  anterior,  gracias  a  la función  de  intercalación  personalizada,  el  cargador  de  datos  puede  crear  lotes  de  diferentes  longitudes.  En  la siguiente  sección, se cargará  un  LLM  preentrenado  que se puede ajustar  con  este  cargador  de  datos.


[Carga de un LLM preentrenado](./5_carga_llm_preentrenado.ipynb)